In [1]:
import requests
import xlrd
import pandas as pd
import numpy as np
import json
import time as time_module 
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from shapely.geometry import Point



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

# Retrieving api key
load_dotenv("../key.env")
api_key = os.getenv("API_KEY")
TOKEN = os.getenv('ONEMAPTOKEN')

In [2]:
geospatial_train_path = "../datasets/geospatial_layer/TrainStation_Jul2024/RapidTransitSystemStation.shp"
train_stations = pd.read_excel("../datasets/Train_Stations.xls")
geospatial_train_gdf = gpd.read_file(geospatial_train_path)

### Data-Pre Processing

In [ ]:
# Step 1: Union the geometries for the same station
unioned_gdf = geospatial_train_gdf.dissolve(by='STN_NAM_DE',aggfunc='first')

# Step 2: Calculate the centroid of the unioned polygon
unioned_gdf['centroid'] = unioned_gdf.centroid

# Optional Step: Replace geometry with centroid point
unioned_gdf['geometry'] = unioned_gdf['centroid']

# Reset index to clean up
unioned_gdf.reset_index(inplace=True)

In [ ]:
# Function to normalize station names in train_stations_df
def normalize_station_name(name):
    return name.strip().upper()  # Ensure names are uppercase for consistent merging

# Apply normalization function to train_stations_df
train_stations['Normalized_Station'] = train_stations['MRT_Station'].apply(normalize_station_name)

# Create a column to append " MRT STATION" or " LRT STATION" based on the MRT_Line
train_stations['Station_MRT_LRT'] = train_stations.apply(
    lambda row: f"{row['Normalized_Station']} MRT STATION" if "LRT" not in row['MRT_Line'] else f"{row['Normalized_Station']} LRT STATION",
    axis=1
)

# Apply normalization to geospatial_train_df
# Strip ' MRT STATION' and ' LRT STATION' and normalize to uppercase
unioned_gdf['Normalized_Station'] = unioned_gdf['STN_NAM_DE'].str.strip().str.upper()

# Perform the merge on 'Station_MRT_LRT' from train_stations and 'Normalized_Station' from unioned_gdf
merged_train_stations = train_stations.merge(
    unioned_gdf,
    how='left',
    left_on='Station_MRT_LRT',
    right_on='Normalized_Station'
)

# Keeping necessary columns
columns_to_keep = ['Station_Code', 'MRT_Station', 'MRT_Line', 'TYP_CD_DES', 'geometry']
merged_train_stations = merged_train_stations[columns_to_keep]

# Check the resulting column names and sample data
print(merged_train_stations.head())


In [ ]:
#  Convert Pandas DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_train_stations, geometry='geometry')

#  Reproject the GeoDataFrame to EPSG:4326 (WGS 84 - latitude/longitude)
gdf_4326 = gdf.to_crs(epsg=4326)

# Extract Longitude and Latitude from the reprojected geometries
gdf_4326['Longitude'] = gdf_4326.geometry.x
gdf_4326['Latitude'] = gdf_4326.geometry.y

#  Convert back to a Pandas DataFrame (if you don't need the geometry anymore)
merged_train_stations = pd.DataFrame(gdf_4326)

# Removing redundant columns
columns_to_keep = ['Station_Code', 'MRT_Station', 'MRT_Line', 'Longitude', 'Latitude']
merged_train_stations = merged_train_stations[columns_to_keep]
print(merged_train_stations.head())

In [3]:
### Fetching MRT Routes from OneMapSg

In [ ]:
# Function to fetch rail route from OneMap API
def fetch_route(start, end, date, route_time, mode='RAIL'):
    url = 'https://www.onemap.gov.sg/api/public/routingsvc/route'
    params = {
        'start': start,
        'end': end,
        'routeType': 'pt',
        'date': date,
        'time': route_time,
        'mode': mode,
        'maxWalkDistance': 1000,
        'numItineraries': 3
    }
    headers = {
        'Authorization': f'Bearer {TOKEN}'
    }
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route from {start} to {end}: {e}")
        return None

# Main function to iterate through MRT stations dataframe and fetch the route
def main(mrt_stations_df):
    # Iterate through each station to create routes between consecutive stations
    for i in range(len(mrt_stations_df) - 1):
        start_station = mrt_stations_df.iloc[i]
        end_station = mrt_stations_df.iloc[i + 1]

        start = f"{start_station['Latitude']},{start_station['Longitude']}"
        end = f"{end_station['Latitude']},{end_station['Longitude']}"

        date = '10-29-2024'  # Replace with actual date
        route_time = '12:00:00'  # Set initial time to 12:00 PM

        # Attempt fetching the route, with retries if needed
        for _ in range(5):  # Attempt up to 5 times with increasing time intervals
            data = fetch_route(start, end, date, route_time)
            if data is not None:
                break
            # Increment the time by 30 minutes for the next attempt
            hour, minute, second = map(int, route_time.split(':'))
            minute += 15
            if minute >= 60:
                minute -= 60
                hour += 1
            route_time = f"{hour:02d}:{minute:02d}:{second:02d}"

        # Save route to file if route data is fetched
        if data is not None:
            file_path = f"../datasets/routes/onemapsg_mrt/{start_station['Station_Code']}.json"
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"Generated {file_path}")

            # Delay to avoid API rate limiting
            time_module.sleep(1)
        else:
            print(f"Failed to fetch route between {start_station['MRT_Station']} and {end_station['MRT_Station']}")

# Run the main function
main(merged_train_stations)

In [ ]:
#Fetching Routes of TEL4 